In [20]:
import nltk

nltk.download('conll2002')
print(nltk.corpus.conll2002.fileids())

[nltk_data] Downloading package conll2002 to /home/s2-rex/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!
['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']


In [17]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

print(test_sents[:3])

[[('La', 'DA', 'B-LOC'), ('Coruña', 'NC', 'I-LOC'), (',', 'Fc', 'O'), ('23', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFECOM', 'NP', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], [('Las', 'DA', 'O'), ('reservas', 'NC', 'O'), ('"', 'Fe', 'O'), ('on', 'NC', 'O'), ('line', 'AQ', 'O'), ('"', 'Fe', 'O'), ('de', 'SP', 'O'), ('billetes', 'NC', 'O'), ('aéreos', 'AQ', 'O'), ('a', 'SP', 'O'), ('través', 'NC', 'O'), ('de', 'SP', 'O'), ('Internet', 'NC', 'B-MISC'), ('aumentaron', 'VMI', 'O'), ('en', 'SP', 'O'), ('España', 'VMN', 'B-LOC'), ('un', 'DI', 'O'), ('300', 'Z', 'O'), ('por', 'SP', 'O'), ('ciento', 'PN', 'O'), ('en', 'SP', 'O'), ('el', 'DA', 'O'), ('primer', 'AO', 'O'), ('trimestre', 'NC', 'O'), ('de', 'SP', 'O'), ('este', 'DD', 'O'), ('año', 'NC', 'O'), ('con', 'SP', 'O'), ('respecto', 'NC', 'O'), ('al', 'SP', 'O'), ('mismo', 'DI', 'O'), ('período', 'NC', 'O'), ('de', 'SP', 'O'), ('1999', 'Z', 'O'), (',', 'Fc', 'O'), ('aseguró', 'VMI', 'O'), ('hoy', 'RG', 

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'word[-3:]': word[-3:],
            'word[-2:]': word[-2:],
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'postag': postag,
            'postag[:2]': postag[:2],
        }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
            word1 = sent[i+1][0]
            postag1 = sent[i+1][1]
            features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag[:2]': postag1[:2],
            })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

ลองทดสอบกับข้อความประโยคแรก

In [4]:
import json

print(json.dumps(sent2features(train_sents[0])[0], indent=2))

{
  "bias": 1.0,
  "word.lower()": "melbourne",
  "word[-3:]": "rne",
  "word[-2:]": "ne",
  "word.isupper()": false,
  "word.istitle()": true,
  "word.isdigit()": false,
  "postag": "NP",
  "postag[:2]": "NP",
  "BOS": true,
  "+1:word.lower()": "(",
  "+1:word.istitle()": false,
  "+1:word.isupper()": false,
  "+1:postag": "Fpa",
  "+1:postag[:2]": "Fp"
}


ต่อไปเราจะ extract features ข้อมูลทั้งหมด

In [5]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

เริ่ม train กันเลย เราใช้ algorithm L-BFGS กับ Elastic Net (L1+L2) regularization

In [14]:
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 11.3min finished


best params: {'c1': 0.15929842918468004, 'c2': 0.02277912266999813}
best CV score: 0.7493753472309697
model size: 1.20M


การ evaluation ผลโมเดล เนื่องจากเราต้องประเมินทุก label ยกเว้น O แล้วคำนวณ F1 score และดูรายละเอียดทั้งหมดจาก classification report


In [15]:
from sklearn_crfsuite import metrics

# ใช้ parameters ที่ดีที่สุด
crf = rs.best_estimator_

labels = list(crf.classes_)
labels.remove('O')
print(labels)

#F1 score
y_pred = crf.predict(X_test)
print(metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels))

#group B and I results
#classification report
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

['B-LOC', 'B-ORG', 'B-PER', 'I-PER', 'B-MISC', 'I-ORG', 'I-LOC', 'I-MISC']
0.7973682955279361
             precision    recall  f1-score   support

      B-LOC      0.807     0.785     0.796      1084
      I-LOC      0.686     0.646     0.666       325
     B-MISC      0.739     0.552     0.632       339
     I-MISC      0.715     0.576     0.638       557
      B-ORG      0.809     0.837     0.823      1400
      I-ORG      0.849     0.797     0.822      1104
      B-PER      0.843     0.886     0.864       735
      I-PER      0.897     0.943     0.919       634

avg / total      0.810     0.788     0.797      6178

